In [96]:
import pandas as pd

# Load the dataset
features = pd.read_csv('features.csv')

# Print the first few rows of the dataset
print(features.head())


   Store        Date  Temperature  Fuel_Price  MarkDown1  MarkDown2   
0      1  2010-02-05        42.31       2.572        NaN        NaN  \
1      1  2010-02-12        38.51       2.548        NaN        NaN   
2      1  2010-02-19        39.93       2.514        NaN        NaN   
3      1  2010-02-26        46.63       2.561        NaN        NaN   
4      1  2010-03-05        46.50       2.625        NaN        NaN   

   MarkDown3  MarkDown4  MarkDown5         CPI  Unemployment  IsHoliday  
0        NaN        NaN        NaN  211.096358         8.106      False  
1        NaN        NaN        NaN  211.242170         8.106       True  
2        NaN        NaN        NaN  211.289143         8.106      False  
3        NaN        NaN        NaN  211.319643         8.106      False  
4        NaN        NaN        NaN  211.350143         8.106      False  


In [97]:
# Check for missing values in each column
missing_values = features.isnull().sum()
print(missing_values)


Store              0
Date               0
Temperature        0
Fuel_Price         0
MarkDown1       4158
MarkDown2       5269
MarkDown3       4577
MarkDown4       4726
MarkDown5       4140
CPI              585
Unemployment     585
IsHoliday          0
dtype: int64


In [98]:
# Identify non-numeric columns
non_numeric_cols = features.select_dtypes(include=['object']).columns
print("Non-numeric columns:")
print(non_numeric_cols)


Non-numeric columns:
Index(['Date'], dtype='object')


In [99]:
# Attempt to convert non-numeric columns to numeric, coercing errors to NaN
features[non_numeric_cols] = features[non_numeric_cols].apply(pd.to_numeric, errors='coerce')

# Convert the 'Date' column to datetime format
features['Date'] = pd.to_datetime(features['Date'], errors='coerce')


In [102]:
# Fill missing values with the mean of each column
features_filled_mean = features.fillna(features.mean())

# Or fill missing values with the median of each column
features_filled_median = features.fillna(features.median())

placeholder_date = pd.Timestamp('2000-01-01')  # Define a placeholder date
features['Date'] = features['Date'].fillna(placeholder_date)



In [103]:
# Verify that there are no missing values left
print(features_filled_mean.isnull().sum())


Store           0
Date            0
Temperature     0
Fuel_Price      0
MarkDown1       0
MarkDown2       0
MarkDown3       0
MarkDown4       0
MarkDown5       0
CPI             0
Unemployment    0
IsHoliday       0
dtype: int64


In [91]:
import pandas as pd

# Load the dataset
stores = pd.read_csv('stores.csv')

# Print the first few rows of the dataset
print(stores.head())

   Store Type    Size
0      1    A  151315
1      2    A  202307
2      3    B   37392
3      4    A  205863
4      5    B   34875


In [106]:
# Check for missing values
missing_values = stores.isnull().sum()
print("Missing values in each column:")
print(missing_values)


Missing values in each column:
Store    0
Type     0
Size     0
dtype: int64


In [93]:
import pandas as pd

# Load the dataset
train = pd.read_csv('train.csv')

# Print the first few rows of the dataset
print(train.head())

   Store  Dept        Date  Weekly_Sales  IsHoliday
0      1     1  2010-02-05      24924.50      False
1      1     1  2010-02-12      46039.49       True
2      1     1  2010-02-19      41595.55      False
3      1     1  2010-02-26      19403.54      False
4      1     1  2010-03-05      21827.90      False


In [94]:
# Check for missing values
missing_values = train.isnull().sum()
print("Missing values in each column:")
print(missing_values)

Missing values in each column:
Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
dtype: int64


In [129]:
import pandas as pd

# Load the datasets
features = pd.read_csv('features.csv')
stores = pd.read_csv('stores.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Combine the datasets
combined = pd.concat([features, stores, train, test], axis=1)

In [130]:
combined.columns = combined.columns.str.strip()  # Remove any leading/trailing whitespace
combined = combined.loc[:, ~combined.columns.duplicated()]  # Remove duplicate columns


In [132]:
numeric_features = combined.select_dtypes(include=['float64', 'int64'])


# Compute IQR for outlier detection
Q1 = numeric_features.quantile(0.25)
Q3 = numeric_features.quantile(0.75)
IQR = Q3 - Q1

# Check for outliers
outliers = (numeric_features < (Q1 - 1.5 * IQR)) | (numeric_features > (Q3 + 1.5 * IQR))
outliers_mask = ~outliers.any(axis=1)

# Display the number of outliers in each column
print("Number of outliers in each column:")
print(outliers.sum())

# Ensure that the boolean mask aligns with the DataFrame index
print("Length of mask:", len(outliers_mask))
print("Length of combined DataFrame:", len(combined))

# Remove outliers from the combined dataset
combined_no_outliers = combined[outliers_mask]

# Display the cleaned dataset
print("Cleaned dataset:")
print(combined_no_outliers.head())


Number of outliers in each column:
Store               0
Temperature         7
Fuel_Price          0
MarkDown1         237
MarkDown2         436
MarkDown3         480
MarkDown4         337
MarkDown5         212
CPI                 0
Unemployment      386
Size                0
Dept                0
Weekly_Sales    35521
dtype: int64
Length of mask: 421570
Length of combined DataFrame: 421570
Cleaned dataset:
   Store        Date  Temperature  Fuel_Price  MarkDown1  MarkDown2   
0    1.0  2010-02-05        42.31       2.572        NaN        NaN  \
1    1.0  2010-02-12        38.51       2.548        NaN        NaN   
2    1.0  2010-02-19        39.93       2.514        NaN        NaN   
3    1.0  2010-02-26        46.63       2.561        NaN        NaN   
4    1.0  2010-03-05        46.50       2.625        NaN        NaN   

   MarkDown3  MarkDown4  MarkDown5         CPI  Unemployment IsHoliday Type   
0        NaN        NaN        NaN  211.096358         8.106     False    A  \
1   

In [133]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Impute missing values
numeric_data = combined_no_outliers.select_dtypes(include=['float64', 'int64'])
categorical_data = combined_no_outliers.select_dtypes(include=['object'])

numeric_imputer = SimpleImputer(strategy='median')
categorical_imputer = SimpleImputer(strategy='most_frequent')

numeric_data_imputed = numeric_imputer.fit_transform(numeric_data)
categorical_data_imputed = categorical_imputer.fit_transform(categorical_data)

# Convert imputed data back to DataFrame
numeric_data_imputed_df = pd.DataFrame(numeric_data_imputed, columns=numeric_data.columns, index=combined_no_outliers.index)
categorical_data_imputed_df = pd.DataFrame(categorical_data_imputed, columns=categorical_data.columns, index=combined_no_outliers.index)

# Concatenate the imputed data
combined_imputed = pd.concat([numeric_data_imputed_df, categorical_data_imputed_df], axis=1)

# Scale numerical features
scaler = StandardScaler()
numerical_features = combined_imputed.select_dtypes(include=['float64', 'int64'])
numerical_scaled = scaler.fit_transform(numerical_features)

numerical_scaled_df = pd.DataFrame(numerical_scaled, columns=numerical_features.columns, index=combined_imputed.index)
combined_scaled = pd.concat([numerical_scaled_df, categorical_data_imputed_df], axis=1)

# One-hot encode categorical variables
combined_encoded = pd.get_dummies(combined_scaled, drop_first=True)

# Feature Engineering example (create interaction features)
combined_encoded['Temperature_Fuel_Price'] = combined_encoded['Temperature'] * combined_encoded['Fuel_Price']

# Verify final dataset
print(combined_encoded.head())


       Store  Temperature  Fuel_Price  MarkDown1  MarkDown2  MarkDown3   
0 -12.993058    -8.680605  -15.525258  -0.018277   -0.02768  -0.031209  \
1 -12.993058   -10.306192  -15.932429  -0.018277   -0.02768  -0.031209   
2 -12.993058    -9.698736  -16.509255  -0.018277   -0.02768  -0.031209   
3 -12.993058    -6.832569  -15.711878  -0.018277   -0.02768  -0.031209   
4 -12.993058    -6.888181  -14.626089  -0.018277   -0.02768  -0.031209   

   MarkDown4  MarkDown5       CPI  Unemployment  ...  Date_2013-06-21   
0  -0.025918    -0.0229  4.212300      2.361631  ...            False  \
1  -0.025918    -0.0229  4.240470      2.361631  ...            False   
2  -0.025918    -0.0229  4.249545      2.361631  ...            False   
3  -0.025918    -0.0229  4.255437      2.361631  ...            False   
4  -0.025918    -0.0229  4.261330      2.361631  ...            False   

   Date_2013-06-28  Date_2013-07-05  Date_2013-07-12  Date_2013-07-19   
0            False            False        

In [134]:
# Check for necessary columns
required_columns = ['Price', 'Units_Sold', 'Revenue']
missing_columns = [col for col in required_columns if col not in combined.columns]

if missing_columns:
    print(f"Missing columns: {missing_columns}")
else:
    print("All necessary columns are present.")

Missing columns: ['Price', 'Units_Sold', 'Revenue']


In [139]:
combined_no_outliers.columns


Index(['Store', 'Date', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2',
       'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment',
       'IsHoliday', 'Type', 'Size', 'Dept', 'Weekly_Sales'],
      dtype='object')